In [1]:
import psycopg2
from assets.classTypes import Category
from assets.scrape import UseBeautifulSoup as useScrape
from assets.adScrape import advertisementScrape as useAdScrape
from assets.pagination import createLinkList as createLinkList

initialUrl = 'https://www.zangia.mn/'
categorySet = set()
adUrlDict = {}
adsSet = set()
conn = None
cursor = None

In [50]:
create_category_script = '''CREATE TABLE IF NOT EXISTS category(
    _id varchar NOT NULL UNIQUE,
    url varchar NOT NULL,
    name varchar NOT NULL,
    parent_id varchar NOT NULL,
    PRIMARY KEY (_id),
    FOREIGN KEY (parent_id) REFERENCES category(_id)
);'''

create_advertisement_script = '''CREATE TABLE IF NOT EXISTS advertisement(
    _id varchar NOT NULL UNIQUE,
    category_id varchar NOT NULL,
    url varchar NOT NULL,
    company varchar,
    title varchar,
    roles varchar,
    requirements varchar,
    additionalInfo varchar,
    city varchar,
    district varchar,
    exactAddress varchar,
    level varchar,
    type varchar,
    minSalary float,
    maxSalary float,
    isDealable bool,
    phoneNumber varchar,
    publishedDate varchar,
    PRIMARY KEY (_id),
    FOREIGN KEY (category_id) REFERENCES category(_id)
);'''
insert_category_script = 'INSERT INTO category (url, name) VALUES (%s, %s)'

In [31]:
soup = useScrape(initialUrl)
navigatorList = soup.find_all('div', class_='filter')
for navigator in navigatorList:
    if navigator.find('h3').text.strip() != 'Салбар, мэргэжил':
        continue
    # ALL CATEGORY LINKS
    categoryList = navigator.find_all('div')


for categoryItem in categoryList:
    categories = categoryItem.find('a')
    url = initialUrl + categories['href']
    tempCategory = Category(url, categories.text)
    print('CATEGORY LINK SCRAPED! ', url)
    soup = useScrape(url)
    subCategory = soup.find('div', class_='pros')
    # ALL SUBCATEGORY LINKS
    subCategoryList = subCategory.find_all('a')
    for subCategoryItem in subCategoryList:
        subCategoryUrl = initialUrl + subCategoryItem['href']
        tempSubCategory = Category(
            subCategoryUrl, subCategoryItem.text, tempCategory)
        categorySet.add(tempSubCategory)
    categorySet.add(tempCategory)

CATEGORY LINK SCRAPED!  https://www.zangia.mn/job/list/b.17
CATEGORY LINK SCRAPED!  https://www.zangia.mn/job/list/b.33
CATEGORY LINK SCRAPED!  https://www.zangia.mn/job/list/b.19
CATEGORY LINK SCRAPED!  https://www.zangia.mn/job/list/b.20
CATEGORY LINK SCRAPED!  https://www.zangia.mn/job/list/b.22
CATEGORY LINK SCRAPED!  https://www.zangia.mn/job/list/b.11
CATEGORY LINK SCRAPED!  https://www.zangia.mn/job/list/b.40
CATEGORY LINK SCRAPED!  https://www.zangia.mn/job/list/b.12
CATEGORY LINK SCRAPED!  https://www.zangia.mn/job/list/b.36
CATEGORY LINK SCRAPED!  https://www.zangia.mn/job/list/b.34
CATEGORY LINK SCRAPED!  https://www.zangia.mn/job/list/b.4
CATEGORY LINK SCRAPED!  https://www.zangia.mn/job/list/b.31
CATEGORY LINK SCRAPED!  https://www.zangia.mn/job/list/b.39
CATEGORY LINK SCRAPED!  https://www.zangia.mn/job/list/b.13
CATEGORY LINK SCRAPED!  https://www.zangia.mn/job/list/b.41
CATEGORY LINK SCRAPED!  https://www.zangia.mn/job/list/b.26
CATEGORY LINK SCRAPED!  https://www.zangi

In [61]:
try:
  conn = psycopg2.connect(database="chatbot",
                        user='chatbot', password='1234!@#$',
                        host='3.228.127.116', port='5432'
                        )
  conn.autocommit = True
  cursor = conn.cursor()
  cursor.execute(create_category_script)
  cursor.execute(create_advertisement_script)
  
  def upsertCategory (cat: Category):
    if (cat.parentCategory != None):
      upsertCategory(cat.parentCategory)
      
    r = cursor.execute("select * from categry where external_id = ?", cat.externalId)
    if (r != None):
      cursor.execute('update category set name = ? where id = ?', r.name, r.id)
    else:
      id = cursor.execute('insert')
      cat.id = id
  for categoryItem in categorySet:
    # insert_value = (categoryItem.url, categoryItem.name)
    # cursor.execute(insert_category_script, insert_value)
    upsertCategory(categoryItem)

  conn.commit()
  

except Exception as error:
  print(error)

finally:
  if cursor is not None:
    cursor.close()
  if conn is not None:
    conn.close()

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
